In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from common import data
from common.training import training_loop, get_predictions

from algos import stoc2 as stoc
from algos import cbl

from common.models import simple_conv
from common import metrics
from common.models.modutils import load_model


In [5]:
log_dir = "./outs/stoc_2"

horizon = 1024
stride = 512 + 128

batch_size = 8

num_epochs = list(range(1, 5))
out_feats = 64
k = 5

learning_rate = 1e-3

weight_decay = 1e-1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [6]:
"""
../data/r216n09.gzip - 16388 timesteps - 3.76% anomalies - len seq anomal 4.9
../data/r229n10.gzip - 13239 timesteps - 4.21% anomalies - len seq anomal 4.6
../data/r233n14.gzip - 15384 timesteps - 3.66% anomalies - len seq anomal 4.8
"""
train = ["../data/r216n09.gzip", "../data/r229n10.gzip"]
test = ["../data/r233n14.gzip"]
m_data_train = data.Marconi100Dataset(train, scaling=data.Scaling.STANDARD)
m_data_test = data.Marconi100Dataset(test, scaling=data.Scaling.STANDARD)


Loading: 100%|██████████| 1/1 [00:04<00:00,  4.80s/it]


In [7]:
train_dataset = data.UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)
test_dataset = data.UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

print(len(train_dataset), len(test_dataset))

47 24


In [8]:
base_width = 128
down_layers = 2
dilations = [False, True]

backbone = simple_conv.Encoder(data.NUM_FEATURES, down_layers, base_width, dilations)
model = simple_conv.SimpleConv(backbone, out_feats, horizon)

optim = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
lr_sched = CosineAnnealingLR(optim, sum(num_epochs))

aug = cbl.pipeline(
    cbl.random_apply(cbl.left_to_right_flipping(1), 0.5),
    cbl.random_apply(cbl.crop_and_resize(1.5, 3), 0.8),
)

engine = cbl.CBLEngine(
    model=model,
    optimizer=optim,
    temperature=0.5,
    aug_1=aug,
    aug_2=aug,
    lr_scheduler=lr_sched,
    device=device,
)

stoc_engine = stoc.STOC(engine)


In [9]:
with SummaryWriter(log_dir + "/logs") as writer:
    stoc_engine.fit(
        train_dataset,
        test_dataset,
        batch_size,
        k,
        num_epochs,
        writer=writer,
        save_path=log_dir + "/models",
    )


# stoc_engine.fit(train_dataset, test_dataset, batch_size, k, num_epochs)

f:\Users\Zumo\Documents\Github\AI_Industry_Project\notebooks\algos\stoc2.py:72: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  score_distr = np.array(
f:\Users\Zumo\Documents\Github\AI_Industry_Project\notebooks\algos\stoc2.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  score_distr = np.array(


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
cmodel = metrics.default_cmodel()


In [ ]:
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)
scores, labels = get_predictions(stoc_engine.predict, test_loader)


In [ ]:
cmodel.fit(scores, labels).optimize()


In [ ]:
metrics.plot_cost(cmodel)


In [ ]:
metrics.plot_errors_curve(
    cmodel.false_positives, cmodel.false_negatives, cmodel.thresholds, figsize=(15, 5)
)


In [ ]:
pr, rec, f1 = metrics.precision_recall_f1(
    cmodel.false_positives, cmodel.false_negatives
)
metrics.plot_precision_recall_f1_curve(pr, rec, f1, cmodel.thresholds)
